# Setting-up environment

In [1]:
!cp -r ../input/updated-code-3-2/* /kaggle/working/

In [2]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 6.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 51.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 37.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.9/466.9 kB 34.0 MB/s eta 0:00:00
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=464219 sha256=8f17f043854d4a416625f32b9a06c5d407d7547ebef10150242978b9c48afb62
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
  Created wheel for jupyter-contrib-nbextensions: filename=jupyter_contrib_nbextensions-0.7.0-py2.py3-none-any.whl size=23428781 sha256=ed22ff061a9fd78d4b3d0cbe6a01ae819bb63acc21b79870de12203747a81a32
  Stored in directory: /root/.cache/pip/wheels/ea/cc/7

In [1]:
!pip install lightfm tqdm optuna ipykernel matplotlib implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for implicit from https://files.pythonhosted.org/packages/cd/cc/deac70cae8cc32c9885d0cd73bc66e1b3cbea36ae7080b8c83995eaf5322/implicit-0.7.2-cp310-cp310-manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 95.1 MB/s eta 0:00:00:00:0100:01
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=448469 sha256=856dc8352290bb6169203901af59488555550903a2a88321667115c1430199b4
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [4]:
!python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorization_Cython_Epoch.c:1109:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorization_Cython_Epoch.c: In function ‘__pyx_pf_32MatrixFactorization

# Imports

In [2]:
import time
import optuna
import numpy as np
from numpy import linalg as LA
import pandas as pd
import scipy.sparse as sps
from scipy.sparse import vstack, csr_matrix
from scipy.stats import loguniform
from lightfm import LightFM
from lightfm.evaluation import auc_score, precision_at_k
import matplotlib.pyplot as plt
from multiprocessing import cpu_count

In [3]:
from Recommenders.BaseRecommender import BaseRecommender
from Recommenders.BaseMatrixFactorizationRecommender import BaseMatrixFactorizationRecommender
from Recommenders.NonPersonalizedRecommender import TopPop

#---- CF
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

#---- Matrix Factorization
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#---- CF w/ ML
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender, MultiThreadSLIM_SLIMElasticNetRecommender
from implicit.als import AlternatingLeastSquares

#---- Others
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.Recommender_utils import check_matrix

In [4]:
seed = 69
np.random.seed(seed)

# Data Import and Preprocessing

In [5]:
# Import training data
URM_path = "../input/data-books/data_train.csv"
URM_all_dataframe = pd.read_csv(filepath_or_buffer=URM_path,
                                header=0,
                                dtype={0:int, 1:int, 2:int},
                                engine='python')

URM_all_dataframe.columns = ["user_id", "item_id", "interaction"]

In [6]:
# Import target users
target_path = "../input/data-books/data_target_users_test.csv"
target_dataframe= pd.read_csv(filepath_or_buffer=target_path,
                                header=0,
                                dtype={0:int},
                                engine='python')
target_dataframe.columns = ["user_id"]
target_dataframe

,user_id
0,1
1,2
2,3
3,4
4,5
...,...
10877,13020
10878,13021
10879,13022
10880,13023


In [7]:
def preprocess_data(ratings: pd.DataFrame):
    unique_users = ratings.user_id.unique()
    unique_items = ratings.item_id.unique()

    num_users, min_user_id, max_user_id = unique_users.size, unique_users.min(), unique_users.max()
    num_items, min_item_id, max_item_id = unique_items.size, unique_items.min(), unique_items.max()

    print(num_users, min_user_id, max_user_id)
    print(num_items, min_item_id, max_item_id)

    mapping_user_id = pd.DataFrame({"mapped_user_id": np.arange(num_users), "user_id": unique_users})
    mapping_item_id = pd.DataFrame({"mapped_item_id": np.arange(num_items), "item_id": unique_items})

    ratings = pd.merge(left=ratings,
                       right=mapping_user_id,
                       how="inner",
                       on="user_id")

    ratings = pd.merge(left=ratings,
                       right=mapping_item_id,
                       how="inner",
                       on="item_id")

    return ratings

In [8]:
# Call preprocess data function
ratings = preprocess_data(URM_all_dataframe)

12638 1 13024
22222 1 22347


## Conversion to Sparse Matrix

In [9]:
URM = sps.coo_matrix((ratings.interaction.values, (ratings.mapped_user_id.values, ratings.mapped_item_id.values)))

## Data Splits

In [10]:
urm_train, urm_test = split_train_in_two_percentage_global_sample(URM, train_percentage = 0.80)
urm_train, urm_validation = split_train_in_two_percentage_global_sample(urm_train, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(urm_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2630 (20.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2127 (16.8%) Users that have less than 1 test interactions


# **Custom Models**

In [11]:
class ScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1*alpha + R2*(1-alpha)

    """

    RECOMMENDER_NAME = "ScoresHybridRecommender"

    def __init__(self, URM_train, recommender_1, recommender_2):
        super(ScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2


    def fit(self, alpha=0.5):
        self.alpha = alpha

    def _compute_item_score(self, user_id_array, items_to_compute=None):

        # In a simple extension this could be a loop over a list of pretrained recommender objects
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)

        item_weights = item_weights_1*self.alpha + item_weights_2*(1-self.alpha)

        return item_weights

In [12]:
class DifferentLossScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1/norm*alpha + R2/norm*(1-alpha) where R1 and R2 come from
    algorithms trained on different loss functions.

    """

    RECOMMENDER_NAME = "DifferentLossScoresHybridRecommender"


    def __init__(self, URM_train, recommender_1, recommender_2):
        super(DifferentLossScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        
        
        
    def fit(self, norm, alpha = 0.5):

        self.alpha = alpha
        self.norm = norm


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)

        norm_item_weights_1 = LA.norm(item_weights_1, self.norm)
        norm_item_weights_2 = LA.norm(item_weights_2, self.norm)
        
        
        if norm_item_weights_1 == 0:
            raise ValueError("Norm {} of item weights for recommender 1 is zero. Avoiding division by zero".format(self.norm))
        
        if norm_item_weights_2 == 0:
            raise ValueError("Norm {} of item weights for recommender 2 is zero. Avoiding division by zero".format(self.norm))
        
        item_weights = item_weights_1 / norm_item_weights_1 * self.alpha + item_weights_2 / norm_item_weights_2 * (1-self.alpha)

        return item_weights

In [13]:
class LightFMCFRecommender(BaseRecommender):
    """LightFMCFRecommender"""

    RECOMMENDER_NAME = "LightFMCFRecommender"

    def __init__(self, URM_train):
        super(LightFMCFRecommender, self).__init__(URM_train)
    
    
    def fit(self, epochs = 300, user_alpha=1e-6, item_alpha = 1e-6, n_factors = 10, n_threads = 4, max_sampled=3, loss='warp', learning_schedule='adagrad'):
        
        # Let's fit a WARP model
        self.lightFM_model = LightFM(loss=loss,
                                     user_alpha=user_alpha,
                                     item_alpha=item_alpha,
                                     no_components=n_factors,
                                     max_sampled=max_sampled,
                                     learning_schedule=learning_schedule)

        self.lightFM_model = self.lightFM_model.fit(self.URM_train, 
                                       epochs=epochs,
                                       num_threads=n_threads,
                                       verbose=True)


    def _compute_item_score(self, user_id_array, items_to_compute = None):
        
        # Create a single (n_items, ) array with the item score, then copy it for every user
        items_to_compute = np.arange(self.n_items)
        
        item_scores = - np.ones((len(user_id_array), self.n_items)) * np.inf

        for user_index, user_id in enumerate(user_id_array):
            item_scores[user_index] = self.lightFM_model.predict(int(user_id), 
                                                                 items_to_compute)

        return item_scores

In [14]:
class GeneralizedLinearHybridRecommender(BaseRecommender):
    """
    This recommender merges N recommendes by weighting their ratings
    """

    RECOMMENDER_NAME = "GeneralizedLinearHybridRecommender"

    def __init__(self, URM_train, recommenders: list, verbose=True):
        self.RECOMMENDER_NAME = ''
        for recommender in recommenders:
            self.RECOMMENDER_NAME = self.RECOMMENDER_NAME + recommender.RECOMMENDER_NAME[:-11]
        self.RECOMMENDER_NAME = self.RECOMMENDER_NAME + 'HybridRecommender'

        super(GeneralizedLinearHybridRecommender, self).__init__(URM_train, verbose=verbose)

        self.recommenders = recommenders

    def fit(self, alphas=None):
        self.alphas = alphas

    def save_model(self, folder_path, file_name=None):
        pass

    def _compute_item_score(self, user_id_array, items_to_compute=None):
        result = self.alphas[0]*self.recommenders[0]._compute_item_score(user_id_array,items_to_compute)
        for index in range(1,len(self.alphas)):
            result = result + self.alphas[index]*self.recommenders[index]._compute_item_score(user_id_array,items_to_compute)
        return result

In [15]:
class FastIALSRecommender(BaseMatrixFactorizationRecommender):
    RECOMMENDER_NAME = "FastIALSRecommender"

    AVAILABLE_CONFIDENCE_SCALING = ["linear", "log"]
    
    def __init__(self, URM_train, verbose=True):
        super().__init__(URM_train, verbose=verbose)
        
    def fit(self,
            factors=20,
            regularization=1e-3,
            iterations=100,
            calculate_training_loss=False,
            num_threads=0,
            confidence_scaling='linear',
            alpha=1.0,
            epsilon=0,
            #---- Do not change
            use_native=True,
            use_cg=True,
            use_gpu=True):
        if confidence_scaling not in self.AVAILABLE_CONFIDENCE_SCALING:
           raise ValueError("Value for 'confidence_scaling' not recognized. Acceptable values are {}, provided was '{}'".format(self.AVAILABLE_CONFIDENCE_SCALING, confidence_scaling))

        self.alpha = alpha
        self.epsilon = epsilon
        self.num_factors = factors
        self.reg = regularization
        
        self.USER_factors = self._init_factors(self.n_users, False)  # don't need values, will compute them
        self.ITEM_factors = self._init_factors(self.n_items)
        
        self.recommender = AlternatingLeastSquares(factors=factors, regularization=regularization,
                                                        use_native=use_native, use_cg=use_cg, use_gpu=use_gpu,
                                                        iterations=iterations,
                                                        calculate_training_loss=calculate_training_loss,
                                                        num_threads=num_threads)
        
        self._build_confidence_matrix(confidence_scaling)
        self.recommender.fit(self.C, show_progress=self.verbose)
        
        self.USER_factors = self.recommender.user_factors.to_numpy()
        self.ITEM_factors = self.recommender.item_factors.to_numpy()
        
    
    def _linear_scaling_confidence(self):

        C = check_matrix(self.URM_train, format="csr", dtype = np.float32)
        C.data = 1.0 + self.alpha*C.data

        return C

    def _log_scaling_confidence(self):

        C = check_matrix(self.URM_train, format="csr", dtype = np.float32)
        C.data = 1.0 + self.alpha * np.log(1.0 + C.data / self.epsilon)

        return C
    
    def _build_confidence_matrix(self, confidence_scaling):

        if confidence_scaling == 'linear':
            self.C = self._linear_scaling_confidence()
        else:
            self.C = self._log_scaling_confidence()

        self.C_csc= check_matrix(self.C.copy(), format="csc", dtype = np.float32)
    
    def _init_factors(self, num_factors, assign_values=True):

        if assign_values:
            return self.num_factors**-0.5*np.random.random_sample((num_factors, self.num_factors))

        else:
            return np.empty((num_factors, self.num_factors))

# **Best Model Params**

In [16]:
ItemKNN_params = {
    'topK': 6,
    'shrink': 15,
    'similarity': 'jaccard',
    'normalize': False
}

In [17]:
RP3beta_params = {
    'alpha': 0.307953246083667, 
    'beta': 0.3073797221110665, 
    'topK': 59, 
    'normalize_similarity': True
}

In [18]:
alpha_itemknn_rp3beta=0.7381515719042592

In [19]:
UserKNN_params = {
    'topK': 470,
    'shrink': 0,
    'similarity': 'cosine',
    'normalize': True
}

In [20]:
LightFM_params = {
                  'n_factors': 482,
                  'max_sampled': 5,
                  'user_alpha': 0.00023989649900734266,
                  'item_alpha': 9.740651135253414e-05
                 }

In [21]:
SLIM_params = {
    'l1_ratio': 0.013752256221164005,
    'alpha': 0.0031943927190071775,
    'topK': 622
}

In [22]:
NMF_params = {
    'l1_ratio': 0.005734775635120469,
    'num_factors': 134,
    'beta_loss': 'frobenius',
    'init_type': 'nndsvda',
    'solver': 'multiplicative_update'
}

In [23]:
IALS_params = {
    'confidence_scaling': 'log',
    'epsilon': 0.11624415533664904,
    'factors': 116,
    'regularization': 0.005454427904241962,
    'alpha': 1.7221339971074425
}

## Pairwise tuning

In [24]:
itemKNNCF = ItemKNNCFRecommender(urm_train)
itemKNNCF.fit(**ItemKNN_params)

ItemKNNCFRecommender: URM Detected 462 ( 3.7%) users with no interactions.
ItemKNNCFRecommender: URM Detected 344 ( 1.5%) items with no interactions.
Similarity column 22222 (100.0%), 5649.79 column/sec. Elapsed time 3.93 sec


In [25]:
rp3beta = RP3betaRecommender(urm_train)
rp3beta.fit(**RP3beta_params)

RP3betaRecommender: URM Detected 462 ( 3.7%) users with no interactions.
RP3betaRecommender: URM Detected 344 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 1572.08 column/sec. Elapsed time 14.14 sec


In [26]:
new_similarity = (1 - alpha_itemknn_rp3beta) * itemKNNCF.W_sparse + alpha_itemknn_rp3beta * rp3beta.W_sparse
    
itemKNN_rp3beta = ItemKNNCustomSimilarityRecommender(urm_train)
itemKNN_rp3beta.fit(new_similarity)

ItemKNNCustomSimilarityRecommender: URM Detected 462 ( 3.7%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 344 ( 1.5%) items with no interactions.


In [29]:
slim = MultiThreadSLIM_SLIMElasticNetRecommender(urm_train)
slim.fit(**SLIM_params, workers = int(cpu_count()))

SLIMElasticNetRecommender: URM Detected 462 ( 3.7%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 344 ( 1.5%) items with no interactions.


100%|█████████▉| 22216/22222 [20:47<00:00, 17.81it/s]


In [34]:
!mkdir Models/

In [43]:
slim.save_model("Models/", "SLIM_Elastic_train.zip")

SLIMElasticNetRecommender: Saving model in file 'Models/SLIM_Elastic_train.zip'
DataIO: Folder Models/.temp_46_SLIM_Elastic_train/ already exists, could be the result of a previous failed save attempt or multiple saver are active in parallel. Folder will be removed.
SLIMElasticNetRecommender: Saving complete


In [56]:
slim_2 = MultiThreadSLIM_SLIMElasticNetRecommender(urm_train)
slim_2.load_model("/Models/", "SLIM_Elastic_train.zip")

SLIMElasticNetRecommender: URM Detected 462 ( 3.7%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 344 ( 1.5%) items with no interactions.
SLIMElasticNetRecommender: Loading model from file '/Models/SLIM_Elastic_train.zip'
SLIMElasticNetRecommender: Loading complete


### Tuning Hybrid ItemKNN RP3Beta

In [33]:
class SaveResults(object):

    def __init__(self):
        self.results_df = pd.DataFrame(columns=["result"])

    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]

        # Create a DataFrame from the current trial's results
        trial_df = pd.DataFrame([hyperparam_dict])

        # Use concat instead of append
        self.results_df = pd.concat([self.results_df, trial_df], ignore_index=True)

In [57]:
def objective_function_hybrid_item_slim(optuna_trial):
    alpha = optuna_trial.suggest_float("alpha", 0, 1)
    
    recommender_instance = ScoresHybridRecommender(urm_train, itemKNN_rp3beta, slim_2)
    recommender_instance.fit(alpha)

    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)

    return result_df.loc[10]["MAP"]

In [58]:
optuna_study_hybrid_item_slim = optuna.create_study(direction="maximize")

save_results_hybrid_item_slim = SaveResults()

optuna_study_hybrid_item_slim.optimize(objective_function_hybrid_item_slim,
                      callbacks=[save_results_hybrid_item_slim],
                      n_trials = 300)

[I 2023-12-05 16:12:54,672] A new study created in memory with name: no-name-156d9540-8c30-4a1b-bbf5-6f598fdae8b6


ScoresHybridRecommender: URM Detected 462 ( 3.7%) users with no interactions.
ScoresHybridRecommender: URM Detected 344 ( 1.5%) items with no interactions.
EvaluatorHoldout: Processed 10008 (100.0%) in 16.69 sec. Users per second: 600


[I 2023-12-05 16:13:11,387] Trial 0 finished with value: 0.03141306002816813 and parameters: {'alpha': 0.44930762071559416}. Best is trial 0 with value: 0.03141306002816813.


ScoresHybridRecommender: URM Detected 462 ( 3.7%) users with no interactions.
ScoresHybridRecommender: URM Detected 344 ( 1.5%) items with no interactions.
EvaluatorHoldout: Processed 10008 (100.0%) in 16.56 sec. Users per second: 604


[I 2023-12-05 16:13:27,975] Trial 1 finished with value: 0.03113965177064713 and parameters: {'alpha': 0.8358287023275444}. Best is trial 0 with value: 0.03141306002816813.


...

[I 2023-12-05 17:35:33,993] Trial 297 finished with value: 0.031411049731643485 and parameters: {'alpha': 0.5230720284226571}. Best is trial 51 with value: 0.03147020875363223.


ScoresHybridRecommender: URM Detected 462 ( 3.7%) users with no interactions.
ScoresHybridRecommender: URM Detected 344 ( 1.5%) items with no interactions.
EvaluatorHoldout: Processed 10008 (100.0%) in 16.49 sec. Users per second: 607


[I 2023-12-05 17:35:50,515] Trial 298 finished with value: 0.031376386985649825 and parameters: {'alpha': 0.37210332320676287}. Best is trial 51 with value: 0.03147020875363223.


ScoresHybridRecommender: URM Detected 462 ( 3.7%) users with no interactions.
ScoresHybridRecommender: URM Detected 344 ( 1.5%) items with no interactions.
EvaluatorHoldout: Processed 10008 (100.0%) in 16.60 sec. Users per second: 603


[I 2023-12-05 17:36:07,138] Trial 299 finished with value: 0.03144278228211102 and parameters: {'alpha': 0.5015375586318745}. Best is trial 51 with value: 0.03147020875363223.


In [59]:
pruned_trials = [t for t in optuna_study_hybrid_item_slim.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in optuna_study_hybrid_item_slim.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print("  Number of finished trials: ", len(optuna_study_hybrid_item_slim.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
print("  Value Validation: ", optuna_study_hybrid_item_slim.best_trial.value)

print("Best params:")
print(optuna_study_hybrid_item_slim.best_trial.params)

Study statistics: 
  Number of finished trials:  300
  Number of pruned trials:  0
  Number of complete trials:  300
Best trial:
  Value Validation:  0.03147020875363223
Best params:
{'alpha': 0.509717003821898}


In [ ]:
hybrid_item_slim = ScoresHybridRecommender(urm_train, itemKNN_rp3beta, slim)
hybrid_item_slim.fit(optuna_study_hybrid_item_slim.best_trial.params['alpha'])

In [23]:
alpha_item_slim = 0.509717003821898

## Tuning Hybrid SLIM

In [28]:
slim = MultiThreadSLIM_SLIMElasticNetRecommender(urm_train)
slim.load_model("Models/", "SLIM_Elastic_train.zip")

SLIMElasticNetRecommender: URM Detected 462 ( 3.7%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 344 ( 1.5%) items with no interactions.
SLIMElasticNetRecommender: Loading model from file 'Models/SLIM_Elastic_train.zip'
SLIMElasticNetRecommender: Loading complete


In [36]:
hybrid_item_slim = ScoresHybridRecommender(urm_train, itemKNN_rp3beta, slim)
hybrid_item_slim.fit(alpha_item_slim)

ScoresHybridRecommender: URM Detected 462 ( 3.7%) users with no interactions.
ScoresHybridRecommender: URM Detected 344 ( 1.5%) items with no interactions.


In [30]:
ials = FastIALSRecommender(urm_train)
ials.fit(**IALS_params)

FastIALSRecommender: URM Detected 462 ( 3.7%) users with no interactions.
FastIALSRecommender: URM Detected 344 ( 1.5%) items with no interactions.


  0%|          | 0/100 [00:00<?, ?it/s]

In [37]:
def objective_function_hybrid_ials(optuna_trial):
    alpha = optuna_trial.suggest_float("alpha", 0, 1)
    
    recommender_instance = ScoresHybridRecommender(urm_train, hybrid_item_slim, ials)
    recommender_instance.fit(alpha)

    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)

    return result_df.loc[10]["MAP"]

In [38]:
optuna_study_hybrid_ials = optuna.create_study(direction="maximize")

save_results_hybrid_ials = SaveResults()

optuna_study_hybrid_ials.optimize(objective_function_hybrid_ials,
                      callbacks=[save_results_hybrid_ials],
                      n_trials = 300)

[I 2023-12-05 20:54:18,432] A new study created in memory with name: no-name-be4d462f-c2b4-43bd-a6a6-2c179dd241a8


ScoresHybridRecommender: URM Detected 462 ( 3.7%) users with no interactions.
ScoresHybridRecommender: URM Detected 344 ( 1.5%) items with no interactions.
EvaluatorHoldout: Processed 10008 (100.0%) in 20.26 sec. Users per second: 494


[I 2023-12-05 20:54:38,719] Trial 0 finished with value: 0.027214149315468405 and parameters: {'alpha': 0.5477124334945206}. Best is trial 0 with value: 0.027214149315468405.


ScoresHybridRecommender: URM Detected 462 ( 3.7%) users with no interactions.
ScoresHybridRecommender: URM Detected 344 ( 1.5%) items with no interactions.


/tmp/ipykernel_47/36494528.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results_df = pd.concat([self.results_df, trial_df], ignore_index=True)


...

ScoresHybridRecommender: URM Detected 462 ( 3.7%) users with no interactions.
ScoresHybridRecommender: URM Detected 344 ( 1.5%) items with no interactions.
EvaluatorHoldout: Processed 10008 (100.0%) in 20.30 sec. Users per second: 493


[I 2023-12-05 21:47:11,333] Trial 154 finished with value: 0.031467096164338584 and parameters: {'alpha': 0.9995813405347284}. Best is trial 65 with value: 0.031473495806529604.


ScoresHybridRecommender: URM Detected 462 ( 3.7%) users with no interactions.
ScoresHybridRecommender: URM Detected 344 ( 1.5%) items with no interactions.


[W 2023-12-05 21:47:25,036] Trial 155 failed with parameters: {'alpha': 0.4826653285159985} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_47/3372266972.py", line 7, in objective_function_hybrid_ials
    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)
  File "/kaggle/working/Evaluation/Evaluator.py", line 276, in evaluateRecommender
    results_dict = self._run_evaluation_on_selected_users(recommender_object, self.users_to_evaluate)
  File "/kaggle/working/Evaluation/Evaluator.py", line 476, in _run_evaluation_on_selected_users
    recommended_items_batch_list, scores_batch = recommender_object.recommend(test_user_batch_array,
  File "/kaggle/working/Recommenders/BaseRecommender.py", line 147, in recommend
    scores_batch = self._compute_item_score(user_id_arra

KeyboardInterrupt: 

In [39]:
pruned_trials = [t for t in optuna_study_hybrid_ials.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in optuna_study_hybrid_ials.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print("  Number of finished trials: ", len(optuna_study_hybrid_ials.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
print("  Value Validation: ", optuna_study_hybrid_ials.best_trial.value)

print("Best params:")
print(optuna_study_hybrid_ials.best_trial.params)

Study statistics: 
  Number of finished trials:  156
  Number of pruned trials:  0
  Number of complete trials:  155
Best trial:
  Value Validation:  0.031473495806529604
Best params:
{'alpha': 0.999761270813173}


In [40]:
alpha_hybrid_ials = optuna_study_hybrid_ials.best_trial.params['alpha']

# Test evaluation

In [41]:
urm_train_validation = urm_train + urm_validation

In [42]:
hybrid = ScoresHybridRecommender(urm_train, hybrid_item_slim, ials)
hybrid.fit(alpha_hybrid_ials)

ScoresHybridRecommender: URM Detected 462 ( 3.7%) users with no interactions.
ScoresHybridRecommender: URM Detected 344 ( 1.5%) items with no interactions.


In [43]:
result_df, _ = evaluator_test.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 10511 (100.0%) in 21.35 sec. Users per second: 492


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.080582,0.141487,0.120297,0.038695,0.068053,0.232021,0.126862,0.096514,0.478166,0.296963,...,0.831698,0.39769,0.831698,0.063506,10.27468,0.997837,0.184331,0.792075,1.799773,0.361264


# Submission

In [44]:
urm_full = urm_train+urm_validation+urm_test

In [45]:
top_pop_final = TopPop(urm_full)
top_pop_final.fit()

In [46]:
itemKNNCF = ItemKNNCFRecommender(urm_full)
itemKNNCF.fit(**ItemKNN_params)

Similarity column 22222 (100.0%), 4575.99 column/sec. Elapsed time 4.86 sec


In [47]:
rp3beta = RP3betaRecommender(urm_full)
rp3beta.fit(**RP3beta_params)

RP3betaRecommender: Similarity column 22222 (100.0%), 1373.25 column/sec. Elapsed time 16.18 sec


In [48]:
new_similarity = (1 - alpha_itemknn_rp3beta) * itemKNNCF.W_sparse + alpha_itemknn_rp3beta * rp3beta.W_sparse
    
itemKNN_rp3beta = ItemKNNCustomSimilarityRecommender(urm_full)
itemKNN_rp3beta.fit(new_similarity)

In [49]:
slim = MultiThreadSLIM_SLIMElasticNetRecommender(urm_full)
slim.fit(**SLIM_params, workers = int(cpu_count()))

100%|█████████▉| 22216/22222 [23:42<00:00, 15.61it/s]


In [50]:
slim.save_model("Models/", "SLIM_Elastic_full.zip")

SLIMElasticNetRecommender: Saving model in file 'Models/SLIM_Elastic_full.zip'
SLIMElasticNetRecommender: Saving complete


In [51]:
hybrid_item_slim = ScoresHybridRecommender(urm_full, itemKNN_rp3beta, slim)
hybrid_item_slim.fit(alpha_item_slim)

In [52]:
ials = FastIALSRecommender(urm_train)
ials.fit(**IALS_params)

FastIALSRecommender: URM Detected 462 ( 3.7%) users with no interactions.
FastIALSRecommender: URM Detected 344 ( 1.5%) items with no interactions.


  0%|          | 0/100 [00:00<?, ?it/s]

In [53]:
hybrid = ScoresHybridRecommender(urm_full, hybrid_item_slim, ials)
hybrid.fit(alpha_hybrid_ials)

## Submission preparation

In [54]:
def prepare_submission(ratings: pd.DataFrame, users_to_recommend: np.array, urm_train: sps.csr_matrix, recommender: BaseRecommender, recommendation_length):
    users_ids_and_mappings = ratings[ratings.user_id.isin(users_to_recommend)][["user_id", "mapped_user_id"]].drop_duplicates()
    mapping_to_item_id = dict(zip(ratings.mapped_item_id, ratings.item_id))

    submission = []

    rec_users_arr = users_ids_and_mappings.mapped_user_id.to_numpy()
    recommendations = recommender.recommend(user_id_array= rec_users_arr, cutoff=recommendation_length)

    i = 0

    for idx, row in users_ids_and_mappings.iterrows():
        user_id = row.user_id
        mapped_user_id = row.mapped_user_id

        submission.append((user_id, [mapping_to_item_id[item_id] for item_id in recommendations[i]]))
        i+=1

    return submission

In [55]:
users_to_recommend = target_dataframe.to_numpy().flatten()
users_to_recommend.shape

(10882,)

In [56]:
mapping_to_item_id = dict(zip(ratings.mapped_item_id, ratings.item_id))

In [57]:
urm_ids = np.unique(ratings.user_id)
missing_users = set([i for i in users_to_recommend]) - set([i for i in urm_ids])
missing_users = np.array([x for x in missing_users])

In [58]:
!mkdir Results

In [59]:
def write_submission(submissions, name):
    with open(f"/kaggle/working/Results/{name}.csv", "w") as f:
        f.write("user_id,item_list\n")
        for user_id, items in submissions:
            f.write(f"{user_id},{' '.join([str(item) for item in items])}\n")
        f.close()

In [60]:
submission = prepare_submission(ratings, users_to_recommend, urm_full, hybrid, recommendation_length=10)

In [61]:
rec_missing = top_pop_final.recommend(missing_users, cutoff=10, remove_seen_flag=False)

In [62]:
for user_id in missing_users:
  submission.append((user_id, [mapping_to_item_id[item_id] for item_id in rec_missing[0]]))

In [63]:
(len(submission), len(users_to_recommend))

(10882, 10882)

In [64]:
write_submission(submission, "submission_hybrid_3-fold_ials")

# Testing score ranges

In [30]:
import numpy.linalg as LA
import plotly.express as px

In [65]:
slim_score = slim._compute_item_score(0)
slim_score

array([[0.12441177, 0.46392898, 0.09411763, ..., 0.        , 0.        ,
        0.        ]])

In [70]:
slim_score.shape

(1, 22222)

In [74]:
!pip install plotly

In [77]:
import plotly.express as px
# Let's say 'scores' is your array or list of 22222 scores.
# We create a DataFrame from it:
df = pd.DataFrame({'Score': slim_score[0]})

# Now, we generate an x-axis that corresponds to the index of each score:
df['Index'] = df.index

# Using Plotly to create the scatter plot:
fig = px.scatter(df, x='Index', y='Score')
fig.show()

In [78]:
itemKNN_rp3beta_score = itemKNN_rp3beta._compute_item_score(0)
itemKNN_rp3beta_score

array([[0.02227767, 0.3337021 , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [79]:
# Let's say 'scores' is your array or list of 22222 scores.
# We create a DataFrame from it:
df = pd.DataFrame({'Score': itemKNN_rp3beta_score[0]})

# Now, we generate an x-axis that corresponds to the index of each score:
df['Index'] = df.index

# Using Plotly to create the scatter plot:
fig = px.scatter(df, x='Index', y='Score')
fig.show()

In [80]:
ials_score = ials._compute_item_score(0)
ials_score

array([-0.25467682,  0.5251764 ,  0.15677363, ..., -0.00211788,
        0.00110382,  0.        ], dtype=float32)

In [83]:
# Let's say 'scores' is your array or list of 22222 scores.
# We create a DataFrame from it:
df = pd.DataFrame({'Score': ials_score})

# Now, we generate an x-axis that corresponds to the index of each score:
df['Index'] = df.index

# Using Plotly to create the scatter plot:
fig = px.scatter(df, x='Index', y='Score')
fig.show()

In [87]:
l1_ials = LA.norm([ials_score], 1)
l1_ials_scores = ials_score / l1_ials

l2_ials = LA.norm([ials_score], 2)
l2_ials_scores = ials_score / l2_ials

linf_ials = LA.norm(ials_score, np.inf)
linf_ials_scores = ials_score / linf_ials

lminusinf_ials = LA.norm(ials_score, -np.inf)
lminusinf_ials_scores = ials_score / lminusinf_ials

In [88]:
# Let's say 'scores' is your array or list of 22222 scores.
# We create a DataFrame from it:
df = pd.DataFrame({'Score': l1_ials_scores})

# Now, we generate an x-axis that corresponds to the index of each score:
df['Index'] = df.index

# Using Plotly to create the scatter plot:
fig = px.scatter(df, x='Index', y='Score')
fig.show()

In [89]:
# Let's say 'scores' is your array or list of 22222 scores.
# We create a DataFrame from it:
df = pd.DataFrame({'Score': l2_ials_scores})

# Now, we generate an x-axis that corresponds to the index of each score:
df['Index'] = df.index

# Using Plotly to create the scatter plot:
fig = px.scatter(df, x='Index', y='Score')
fig.show()

In [91]:
# Let's say 'scores' is your array or list of 22222 scores.
# We create a DataFrame from it:
df = pd.DataFrame({'Score': linf_ials_scores})

# Now, we generate an x-axis that corresponds to the index of each score:
df['Index'] = df.index

# Using Plotly to create the scatter plot:
fig = px.scatter(df, x='Index', y='Score')
fig.show()

In [ ]:
# Let's say 'scores' is your array or list of 22222 scores.
# We create a DataFrame from it:
df = pd.DataFrame({'Score': lminusinf_ials_scores})

# Now, we generate an x-axis that corresponds to the index of each score:
df['Index'] = df.index

# Using Plotly to create the scatter plot:
fig = px.scatter(df, x='Index', y='Score')
fig.show()

In [25]:
lightfm = LightFMCFRecommender(urm_train)
lightfm.fit(**LightFM_params, n_threads=int(cpu_count()))

LightFMCFRecommender: URM Detected 462 ( 3.7%) users with no interactions.
LightFMCFRecommender: URM Detected 344 ( 1.5%) items with no interactions.


Epoch: 100%|██████████| 300/300 [04:13<00:00,  1.18it/s]


In [27]:
lightfm_score = lightfm._compute_item_score([0])
lightfm_score

array([[-0.32575092,  0.28988874, -0.43371585, ..., -0.70568401,
        -0.53008372, -0.66958642]])

In [31]:
# Let's say 'scores' is your array or list of 22222 scores.
# We create a DataFrame from it:
df = pd.DataFrame({'Score': lightfm_score[0]})

# Now, we generate an x-axis that corresponds to the index of each score:
df['Index'] = df.index

# Using Plotly to create the scatter plot:
fig = px.scatter(df, x='Index', y='Score')
fig.show()

In [32]:
df.to_csv("lightfm_scores.csv")